In [3]:
import time
from dcicutils import s3Utils
from dcicutils import ff_utils
from core.wfr2 import *
from core.wfr_settings import *

wf_md5 = "md5"
my_env = 'data'
my_auth = ff_utils.get_authentication_with_server({}, ff_env = my_env)

my_s3_util = s3Utils(env=my_env)
raw_bucket = my_s3_util.raw_file_bucket
out_bucket = my_s3_util.outfile_bucket

run_md5 = raw_input("Do you wanna run md5 if missing? (md5/none)")

file_url = '/search/?status=upload%20failed&status=uploading&type=File&limit=all'
#file_url = '/search/?type=FileReference'
all_files = ff_utils.search_metadata(file_url, key=my_auth)
print len(all_files)

printn = 0
counter = 0
for a_file in all_files:  
    counter += 1
    # check for deleted or weird cases
    try:
        if a_file['status'] == 'deleted':
            print "Deleted File", a_file
            continue
    except:
        print a_file
        break
        
    if counter-printn > 100:
        print counter
        printn = counter

    if 'FileProcessed' in a_file['@type']:
            my_bucket = out_bucket
    else:  # covers cases of FileFastq, FileReference, FileMicroscopy
            my_bucket = raw_bucket       
 
    # check if file is in s3
    head_info = my_s3_util.does_key_exist(a_file['upload_key'], my_bucket)
    if not head_info:
        print a_file['accession'], "does not have a file in S3"
        continue
        
    file_id = a_file['accession']
    status = a_file.get('status')
        
    # is there md5sum for gzip
    if not a_file.get('md5sum'):
        print file_id,"does not have the md5sum calculated during upload"
    # Check workflows
    md5_status = 'did_not_run'
    
    # Assumes workflow_runs come in time ordered list, and grabs the last ones for each wf run
    md5_report = get_wfr_out(a_file['uuid'], "md5", my_auth, md_qc = True)
    
    if md5_report['status'] == 'running':
        print(file_id, 'still running')
        continue 

    if status in ["uploading", "upload failed"] and md5_report['status'] != 'complete':
        # if not, shall we run it?
        if run_md5 == 'md5':
            print 'md5 running for', file_id
            inp_f = {'input_file':a_file['@id']}
            run_missing_wfr(step_settings('md5', 'none', 'none'), inp_f, file_id, my_env, tag="")
            print ''
        # user does not want it to be run, so just report
        else:
            print 'md5 run missing for', file_id

    elif status in ["uploading", "upload failed"] and md5_status == 'complete':
        print "ERROR:",file_id ,"There is a successful md5 run but status not switched"
        # if not, shall we run it?
        if run_md5 == 'md5':
            print 'md5 running for', file_id
            inp_f = {'input_file':a_file['@id']}
            run_missing_wfr(step_settings('md5', 'none', 'none'), inp_f, file_id, my_env, tag="")
            print ''
        
    elif md5_status != 'complete':
        if a_file.get('content_md5sum'):
            print('md5run not linked to file')
        else:
            print file_id, status, 'switched status file does not have md5 run or content_md5sum'
            # if not, shall we run it?
            if run_md5 == 'md5':
                print 'md5 running for', file_id
                inp_f = {'input_file':a_file['@id']}
                run_missing_wfr(step_settings('md5', 'none', 'none'), inp_f, file_id, my_env, tag="")
                print ''
            
    print



Do you wanna run md5 if missing? (md5/none)md5
69
object bbce54ce-4a43-41e0-b99b-471e5d3f7a64/4DNFIPE4N3LP not found on bucket elasticbeanstalk-fourfront-webprod-wfoutput
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFIPE4N3LP does not have a file in S3
md5 running for 4DNFIIDEGWLI
about to start run md5_4DNFIIDEGWLI
response from aws was: 
 {u'startDate': datetime.datetime(2018, 7, 9, 12, 52, 33, 949000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '79f5f55c-8398-11e8-a0c3-4943de1977b0', 'HTTPHeaders': {'x-amzn-requestid': '79f5f55c-8398-11e8-a0c3-4943de1977b0', 'content-length': '125', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:md5_4DNFIIDEGWLI'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:md5_4DNFIIDEGWLI

4DNFIMCOUSYJ does not have a file in S3
object f741011d-17db-498d-8cdd-c5bd45a095f3/4DNFICKF4JBU.bw not found on bucket elasticbeanstalk-fourfront-webprod-wfoutput
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFICKF4JBU does not have a file in S3
object 325bc234-41f4-4bd0-be2f-9a2dfe4bb7ee/4DNFICT62ZMH.bw not found on bucket elasticbeanstalk-fourfront-webprod-wfoutput
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFICT62ZMH does not have a file in S3
object ccfff751-4164-459d-9913-3b76b41c7b36/4DNFIE7F58SP.bw not found on bucket elasticbeanstalk-fourfront-webprod-wfoutput
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFIE7F58SP does not have a file in S3
object 2b635989-c8d9-4267-8570-29affbeb9f2a/4DNFIQI3RFQB.bw not found on bucket elasticbeanstalk-fourfront-webprod-wfoutput
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFIQI3RFQB does not have a file in S3
object ae9a8

object c10cf2eb-bb14-45e6-bd08-c0650083e28d/4DNFIKTVL6ZB.tiff not found on bucket elasticbeanstalk-fourfront-webprod-files
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFIKTVL6ZB does not have a file in S3
